In [ ]:
import pysd 
import pandas as pd

In [ ]:
model_file = 'examples/greenhouse.py'
model = pysd.load(model_file)

In [ ]:
df = model.run()
df

In [ ]:
df[['Athmospheric CO2', 'Lower Ocean CO2', 'Upper Ocean CO2',  'Ocean Sedimented', 'Vegetation CO2', 'Soil CO2']]

In [ ]:
# Collect some observations
target_values = {
    'Athmospheric CO2': {
        # From ipcc
        1750: 591,
        2015: 591+279,
    },
    'Athmospheric CO2 Concentraton': {
        # https://www.climate.gov/news-features/understanding-climate/climate-change-atmospheric-carbon-dioxide
        1750: 278,
        1840: 285,
        1915: 300,
        # NOAA: https://climate.nasa.gov/vital-signs/carbon-dioxide/
        1958: 315.23, 1959: 315.98, 1960: 316.91, 1961: 317.64, 1962: 318.45, 1963: 318.99, 1964: 319.62, 1965: 320.04, 1966: 321.37, 1967: 322.18, 1968: 323.05, 1969: 324.62, 1970: 325.68, 1971: 326.32, 1972: 327.46, 1973: 329.68, 1974: 330.19, 1975: 331.13, 1976: 332.03, 1977: 333.84, 1978: 335.42, 1979: 336.84, 1980: 338.76, 1981: 340.12, 1982: 341.48, 1983: 343.15, 1984: 344.87, 1985: 346.35, 1986: 347.61, 1987: 349.31, 1988: 351.69, 1989: 353.20, 1990: 354.45, 1991: 355.70, 1992: 356.55, 1993: 357.22, 1994: 358.96, 1995: 360.97, 1996: 362.74, 1997: 363.88, 1998: 366.84, 1999: 368.54, 2000: 369.71, 2001: 371.32, 2002: 373.45, 2003: 375.98, 2004: 377.70, 2005: 379.98, 2006: 382.09, 2007: 384.03, 2008: 385.83, 2009: 387.64, 2010: 390.10, 2011: 391.85, 2012: 394.06, 2013: 396.74, 2014: 398.81, 2015: 401.01, 2016: 404.41, 2017: 406.76, 2018: 408.71, 2019: 411.65, 2020: 414.21, 2021: 416.41, 2022: 418.53, 2023: 421.99,
    },
    'Ph Ocean': {
        # https://www.eea.europa.eu/ims/ocean-acidification
        1750: 8.25,
        # Autoamtically from https://www.eea.europa.eu/ims/ocean-acidification
        1988: 8.11, 1989: 8.11, 1990: 8.12, 1991: 8.11, 1992: 8.11, 1993: 8.11, 1994: 8.11, 1995: 8.10, 1996: 8.10, 1997: 8.10, 1998: 8.09, 1999: 8.10, 2000: 8.09, 2001: 8.09, 2002: 8.09, 2003: 8.09, 2004: 8.09, 2005: 8.08, 2006: 8.09, 2007: 8.08, 2008: 8.08, 2009: 8.08, 2010: 8.08, 2011: 8.08, 2012: 8.07, 2013: 8.07, 2014: 8.07, 2015: 8.07, 2016: 8.07, 2017: 8.06, 2018: 8.06, 2019: 8.05, 2020: 8.05, 2021: 8.06,
    }        

}

df_target = pd.DataFrame(target_values)

In [ ]:
df.keys()

In [ ]:
# Calculate the error with the model
df_errors = (df_target - df[df_target.columns] )

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
import itertools
colors = itertools.cycle(['r', 'g', 'b', 'c', 'm', 'y', 'k'])

# Plot the data
for col in df_target.columns:

    color = next(colors)

    max_val = max(df[col].max(), df_target[col].max())
    min_val = min(df[col].min(), df_target[col].min())
    norm = lambda x: (x - min_val) / (max_val - min_val)
    ax.plot(norm(df[col]), label=col, color=color)
    ax.scatter(df_target.index, norm(df_target[col]), color=color )

    # add the intial and final values as text 
    ax.text(df.index[0], norm(df[col].iloc[0]), f'{df[col].iloc[0]:.2f}', color=color)
    ax.text(df.index[-1], norm(df[col].iloc[-1]), f'{df[col].iloc[-1]:.2f}', color=color)
ax.legend()
ax.set_ylabel('Normalized value')
    

In [ ]:
df['H Conc']

In [ ]:
from scipy.optimize import minimize
import numpy as np

params_to_optimize = [
    "Upper Ocean CO2", 
    "Alkalinity",
]
def error_function(params):

    model = pysd.load(model_file)
    initial_values = ({param: value for param, value in zip(params_to_optimize, params)})
    final_values =model.run(
        initial_condition=(1750, initial_values),
        return_columns=df_target.columns).iloc[-1]
    target_values = df_target.iloc[0] 
    relative_err =( final_values- target_values) / target_values
    return np.sum(relative_err**2)

In [ ]:
minimize(error_function, [1000, 1000,])

In [ ]:
error_function([1000, 1000])